In [1]:
import pandas as pd
from collections import defaultdict
import dvc.api
from tqdm import tqdm
import numpy as np
from functools import reduce

# get original data

In [2]:
df_original = pd.concat([ 
    pd.read_csv(dvc.api.get_url('modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch1.csv', 
        repo='git@github.com:ContextLogic/multitask-llm-rnd.git')), 
    pd.read_csv(dvc.api.get_url('modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch2.csv', 
        repo='git@github.com:ContextLogic/multitask-llm-rnd.git')),
    pd.read_csv(dvc.api.get_url('modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch3_top74419.csv',
        repo='git@github.com:ContextLogic/multitask-llm-rnd.git'))
])

In [3]:
len(df_original)

324419

In [4]:
len(set(df_original['label_ordering']))

324419

In [39]:
df_original.head(2)

,query,sample_method,label_ordering,top_query_classification_taxonomy
0,dog cat eater,uniform,0,Home & Garden > Pet Products > Cat Supplies > ...
1,fish holder tool,head,18,Sports > Fishing > Fishing Tools


# get attributes

In [5]:
df_attributes = pd.read_csv('../attribute_extraction_metadata_template/attribute_definition_top25L2_filtered_augmented_redacted_addedsinglemulti_01182023.csv')

df_attributes_group = df_attributes.groupby('attribute_field').agg({
    'category_attributevalue': lambda x: reduce(lambda x, y: x+y, [eval(i) for i in x])
}).reset_index()

In [6]:
len(df_attributes_group), df_attributes_group.category_attributevalue.apply(len).sum()

(230, 516378)

In [7]:
attributes = set(df_attributes_group['attribute_field'])

In [8]:
category_paths = set(df_attributes['category'])

In [9]:
more_paths = []
for i in category_paths:
    tmp = []
    for j in i.split(' > '):
        tmp.append(j)
        more_paths.append(' > '.join(tmp))
category_paths.update(more_paths)

# existing valid data

In [32]:
df1 = pd.read_csv('appen/output_batch_correct_v3/query_attribution_till_030923_valid_units.csv')

/tmp/ipykernel_15307/3044835731.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('appen/output_batch_correct_v3/query_attribution_till_030923_valid_units.csv')


In [33]:
del df1['Unnamed: 0']

In [34]:
df2 = pd.read_csv('appen/output_batch_correct_v4/Rework_processed_96,900 units.csv')
df3 = pd.read_csv('appen/output_batch_correct_v4/New Volume_processed_64,136 units.csv')
df4 = pd.read_csv('appen/output_batch_correct_v4/Query Attribution_03.17.23_10k.csv')
df5 = pd.read_csv('appen/output_batch_correct_v4/Query Attribution_03.24.23_3k.csv')

/tmp/ipykernel_15307/2579001723.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('appen/output_batch_correct_v4/Rework_processed_96,900 units.csv')


In [35]:
df = pd.concat([df1, df2, df3, df4, df5])

In [36]:
len(df)

324009

In [37]:
len(set(df['label_ordering']))

324009

In [56]:
df_merge = df.merge(df_original.rename(columns={'query': 'query_original2', 'sample_method': 'sample_method_original', 
                             'top_query_classification_taxonomy': 'top_query_classification_taxonomy_original'}), on='label_ordering')

In [57]:
df_merge.loc[df_merge.query_original.isna(), 'query_original'] = df_merge.loc[df_merge.query_original.isna(), 'query_original2']

In [58]:
(df_merge['sample_method'] == df_merge['sample_method_original']).all()

True

In [59]:
df_merge[df_merge['query'].str.strip() != df_merge['query_original'].str.strip()].sample(2)

,label_ordering,sample_method,l2_category,query_category,rater_output,query,translated_query,no_attributes,unlisted_value,query_original,query_original2,sample_method_original,top_query_classification_taxonomy_original
308738,6435025,uniform,Home & Garden > Home Textile,Home & Garden > Home Textile > Bedding > Blankets,NaN,makita 18 v skins,NaN,True,NaN,makita 18 v skins￼,makita 18 v skins￼,uniform,Home & Garden > Home Textile > Bedding > Blankets
168379,928985,head,"Home & Garden > Arts, Crafts & Sewing","Home & Garden > Arts, Crafts & Sewing > Needle...",NaN,diamond artdallas cowboys,NaN,True,NaN,diamond artdallas cowboys￼,diamond artdallas cowboys￼,head,"Home & Garden > Arts, Crafts & Sewing > Needle..."


In [60]:
(df_merge['query_category'] == df_merge['top_query_classification_taxonomy_original']).all()

True

In [61]:
for i in ['query_original', 'query_original2', 'sample_method_original', 'top_query_classification_taxonomy_original']:
    del df_merge[i]

In [63]:
leftover_ids = set(df_original['label_ordering']) - set(df_merge['label_ordering'])

In [65]:
df_leftover = df_original[df_original.label_ordering.apply(lambda x: x in leftover_ids)]

In [86]:
len(df_leftover)

410

In [69]:
df_leftover.to_csv('appen/output_batch_correct_v5/query_attribution_missed_032423.csv', index=False)

In [71]:
df_merge

,label_ordering,sample_method,l2_category,query_category,rater_output,query,translated_query,no_attributes,unlisted_value
0,225,head,"Home & Garden > Arts, Crafts & Sewing","Home & Garden > Arts, Crafts & Sewing > Button...","Home & Garden > Arts, Crafts & Sewing > Potter...",molde gesso 3d silicone,NaN,NaN,NaN
1,288,head,Jewelry & Accessories > Fine Jewelry,Jewelry & Accessories > Fine Jewelry > Necklaces,Jewelry & Accessories > Fine Jewelry > Necklac...,medayas de oro,NaN,NaN,NaN
2,293,head,Cellphones & Telecommunications > Mobile Phone...,Cellphones & Telecommunications > Mobile Phone...,Cellphones & Telecommunications > Mobile Phone...,case para huawei y9 prime 2019,NaN,NaN,NaN
3,314,uniform,"Home & Garden > Arts, Crafts & Sewing","Home & Garden > Arts, Crafts & Sewing > Appare...","Home & Garden > Arts, Crafts & Sewing > Appare...",lace dots meter,NaN,NaN,NaN
4,694,head,Sports > Fishing,Sports > Fishing > Fishing Rods,Sports > Fishing > Fishing Rods > Fishing Reel...,canne à pêche carnassier,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
324004,4981652,head,Home & Garden > Garden Supplies,Home & Garden > Garden Supplies > Shade > Gazebos,Home & Garden > Garden Supplies > Shade > Gaze...,door canopy uk,NaN,NaN,NaN
324005,4984008,head,Home & Garden > Household Merchandise,Home & Garden > Household Merchandise > Lighte...,Home & Garden > Household Merchandise > Lighte...,glass pipes mini,NaN,NaN,NaN
324006,4986308,head,Home & Garden > Home Storage & Organization,Home & Garden > Home Storage & Organization > ...,NaN,french fry basket,NaN,True,NaN
324007,4986723,head,"Home & Garden > Arts, Crafts & Sewing","Home & Garden > Arts, Crafts & Sewing > Needle...","Home & Garden > Arts, Crafts & Sewing > Needle...",bear diamond painting,NaN,NaN,NaN


In [72]:
correct = []
errors = []
empty = []
for i in df_merge.to_dict('records'):
    res = [] if ((not isinstance(i['rater_output'], str)) and (np.isnan(i['rater_output']))) else i['rater_output'].split('\n')
    if len(res) == 0:
        empty.append(i)
    is_correct = True
    for j in res:
        if len(j.split(' > ')) < 2:
            i['explanation'] = f"{j} has no attribute name"
            errors.append(i)
            is_correct = False
            break
        if j.split(' > ')[-2] not in attributes:
            i['explanation'] = f"{j} has invalid attribute name, because {j.split(' > ')[-2]} is not a valid attribute name"
            errors.append(i)
            is_correct = False
            break
    if is_correct:
        correct.append(i)
len(df), len(correct), len(errors), len(empty), len(correct) + len(errors)

(324009, 324009, 0, 129964, 324009)

In [73]:
len(empty) / len(df_merge)

0.40111231478137954

In [84]:
len(df_merge)

324009

In [85]:
df_merge.to_csv('appen/output_batch_correct_v5/query_attribution_till_032423_valid_units.csv', index=False)